In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import requests

# Define materials of interest and folders to store images and project names
materials = ["Plastic Bottle", "Fabric", "Glass Bottle"]
base_dir = "/content/drive/MyDrive/Material_Project_Data"

# Create main directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Load JSON objects
json_file_path = "/content/drive/MyDrive/DIY_DATASET.json"  # Replace with your actual JSON file path
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Function to create folders for each material
def create_folders(material):
    image_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Images")
    name_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Names")
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(name_folder, exist_ok=True)
    return image_folder, name_folder

# Function to download and save an image
def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f"Failed to download image from {url}: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")

# Iterate through each material
for material in materials:
    image_folder, name_folder = create_folders(material)

    # Iterate through each project in the JSON data
    for project in data:
        project_name = project.get("Product_Name", "Unnamed_Project")
        image_url = project.get("Image_Url", "")
        project_materials = project.get("Materials", [])

        # Check if the material is part of the project materials
        if any(material.lower() in mat.lower() for mat in project_materials):
            # Format the project name to include the material
            formatted_name = f"{material.replace(' ', '_')}_{project_name.replace(' ', '_')}"

            # Save project name as a text file
            project_name_file = os.path.join(name_folder, f"{formatted_name}.txt")
            with open(project_name_file, 'w') as f:
                f.write(f"Project Name: {project_name}\n")
                f.write(f"Webpage URL: {project.get('Webpage_Url', 'N/A')}\n")
                f.write(f"Category: {project.get('Category', 'N/A')}\n")
                f.write(f"Primary Function: {project.get('Primary_Function', 'N/A')}\n")
                f.write(f"Materials: {', '.join(project_materials)}\n")
                f.write(f"Difficulty: {project.get('Difficulty', 'N/A')}\n")
                f.write("\nSteps:\n")
                for step in project.get("Steps", []):
                    f.write(f"Step {step['step']}: {step['description']}\n")

            # Download and save the project image
            if image_url:
                image_file_path = os.path.join(image_folder, f"{formatted_name}.jpg")
                download_image(image_url, image_file_path)


In [ ]:
import os

# Define materials and base directory
materials = ["Plastic Bottle", "Fabric", "Glass Bottle"]
base_dir = "/content/drive/MyDrive/Material_Project_Data"

# Loop through each material
for material in materials:
    # Get the paths for image and project name folders
    image_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Images")
    name_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Names")

    # Loop through each project name file
    for project_name_file in os.listdir(name_folder):
        # Get the corresponding image file path
        project_base_name = os.path.splitext(project_name_file)[0]
        corresponding_image_file = os.path.join(image_folder, f"{project_base_name}.jpg")

        # Check if the image file exists
        if not os.path.exists(corresponding_image_file):
            # If the image is missing, remove the project name file
            project_name_file_path = os.path.join(name_folder, project_name_file)
            os.remove(project_name_file_path)
            print(f"Removed project name file: {project_name_file}")


Removed project name file: Plastic_Bottle_Chalkboard_Fall_Vignette.txt
Removed project name file: Plastic_Bottle_DIY_Gift_Box_for_Kids_(Recycled_Plastic_Bottle).txt
Removed project name file: Plastic_Bottle_Gift_Wrapping_using_Disposable_Water_Bottles.txt
Removed project name file: Plastic_Bottle_Plastic_Bottle_Gift_Box.txt
Removed project name file: Fabric_Spray_Painted_Succulent_Pots.txt
Removed project name file: Fabric_DIY_Photo_Display_Board.txt
Removed project name file: Fabric_Yarn_Wrapped_Hearts_Valentine_Garland.txt
Removed project name file: Fabric_Fabric_and_Wood_Bead_Sunflowers.txt
Removed project name file: Fabric_DIY_Fabric_Flowers_No_Sew_Garland.txt
Removed project name file: Fabric_Painted_Old_Picture_Frames.txt
Removed project name file: Fabric_DIY_Rope_Flower_Pot.txt
Removed project name file: Fabric_Vintage_Linens_Garland.txt
Removed project name file: Fabric_Easy_Lavender_Sachets.txt
Removed project name file: Fabric_Scrap_Fabric_Bowl.txt
Removed project name file: 

In [ ]:
import os
import shutil

# Define paths
dataset_folder = '/content/drive/MyDrive/Images With Captions'
organized_folder = os.path.join(dataset_folder, 'organized_dataset')

# Create the organized_dataset folder if it doesn't exist
if not os.path.exists(organized_folder):
    os.makedirs(organized_folder)

# Iterate over each material folder
for material_folder in os.listdir(dataset_folder):
    material_path = os.path.join(dataset_folder, material_folder)

    if os.path.isdir(material_path):
        # Create a new material folder in the organized dataset
        new_material_path = os.path.join(organized_folder, material_folder)
        if not os.path.exists(new_material_path):
            os.makedirs(new_material_path)

        # Copy all image files into the new material folder
        for image_file in os.listdir(material_path):
            if image_file.endswith(('.jpg', '.png', '.jpeg')):  # Adjust extensions if necessary
                shutil.copy(os.path.join(material_path, image_file), new_material_path)

        # Find the corresponding caption file for this material
        caption_file_name = f"{material_folder.lower().replace(' ', '_')}_captions.txt"
        caption_file_path = os.path.join(dataset_folder, caption_file_name)

        # Check if the caption file exists
        if os.path.isfile(caption_file_path):
            # Read all captions from the main caption file
            with open(caption_file_path, 'r') as f:
                captions = f.readlines()

            # Sort images by file name (assuming names like '00001.jpg')
            images = sorted(os.listdir(material_path))

            # Create individual caption files for each image
            for idx, image_file in enumerate(images):
                if image_file.endswith(('.jpg', '.png', '.jpeg')):  # Ensure it's an image file
                    image_name, _ = os.path.splitext(image_file)

                    # Ensure there's a corresponding caption
                    if idx < len(captions):
                        caption = captions[idx].strip()

                        # Create the individual caption file in the new material folder
                        caption_output_path = os.path.join(new_material_path, f"{image_name}.txt")
                        with open(caption_output_path, 'w') as f:
                            f.write(caption)

print("Organized dataset has been created successfully.")


Organized dataset has been created successfully.


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import spacy
import torchvision.utils as vutils
!pip install -U spacy
!python -m spacy download en_core_web_md


# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.8.0/en_core_web_md-3.8.0-py3-none-any.whl (33.5 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import spacy

# Load SpaCy for text embeddings
nlp = spacy.load("en_core_web_md")  # 300-dimensional embeddings

# Hyperparameters
batch_size = 16
lr = 0.0002
nz = 100  # Noise dimension
num_epochs = 100
image_size = 64  # Resize images to 64x64

# Generator network
class ConditionalGenerator(nn.Module):
    def __init__(self, nz, ngf, nc, text_embedding_dim=300):
        super(ConditionalGenerator, self).__init__()
        self.label_embedding = nn.Linear(text_embedding_dim, nz)

        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz * 2, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, noise, text_embedding):
        text_embedding = self.label_embedding(text_embedding)
        text_embedding = text_embedding.unsqueeze(-1).unsqueeze(-1)
        combined_input = torch.cat((noise, text_embedding), 1)
        return self.main(combined_input)


# Discriminator network
class ConditionalDiscriminator(nn.Module):
    def __init__(self, nc, ndf, text_embedding_dim=300):
        super(ConditionalDiscriminator, self).__init__()
        self.text_embedding_proj = nn.Linear(text_embedding_dim, ndf * 8)

        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.final_layer = nn.Sequential(
            nn.Conv2d(ndf * 8 + ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, image, text_embedding):
        image_features = self.main(image)
        text_features = self.text_embedding_proj(text_embedding)
        text_features = text_features.unsqueeze(-1).unsqueeze(-1).expand_as(image_features)
        combined_features = torch.cat((image_features, text_features), 1)
        return self.final_layer(combined_features)


# Dataset class
class ProjectDataset(Dataset):
    def __init__(self, base_dir, materials, transform):
        self.image_paths = []
        self.texts = []

        # Collect all images and their corresponding project descriptions
        for material in materials:
            image_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Images")
            text_folder = os.path.join(base_dir, f"{material.replace(' ', '_')}_Project_Names")

            image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
            for image_file in image_files:
                image_path = os.path.join(image_folder, image_file)
                project_name = os.path.splitext(image_file)[0]
                text_file = os.path.join(text_folder, f"{project_name}.txt")

                if os.path.exists(text_file):
                    self.image_paths.append(image_path)
                    with open(text_file, 'r') as f:
                        self.texts.append(f.read())

        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load and convert image to RGB
        image = Image.open(self.image_paths[idx]).convert("RGBA")  # Convert palette images to RGBA
        image = image.convert("RGB")  # Ensure all images are in RGB format
        image = self.transform(image)

        # Get text embedding
        text_embedding = torch.tensor(nlp(self.texts[idx]).vector, dtype=torch.float32)

        return image, text_embedding



# Initialize dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

dataset = ProjectDataset(
    base_dir="/content/drive/MyDrive/Material_Project_Data",
    materials=["Plastic Bottle", "Fabric", "Glass Bottle"],
    transform=transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize models, optimizers, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = ConditionalGenerator(nz, 64, 3).to(device)
discriminator = ConditionalDiscriminator(3, 64).to(device)
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, text_embeddings) in enumerate(dataloader):
        real_images = real_images.to(device)
        text_embeddings = text_embeddings.to(device)

        # Train discriminator
        optimizer_d.zero_grad()
        # Real images
        real_labels = torch.ones(real_images.size(0), 1, 1, 1, device=device)
        fake_labels = torch.zeros(real_images.size(0), 1, 1, 1, device=device)

        # Forward pass with real images
        outputs = discriminator(real_images, text_embeddings)
        d_loss_real = criterion(outputs, real_labels)

        # Fake images
        noise = torch.randn(real_images.size(0), nz, 1, 1, device=device)
        fake_images = generator(noise, text_embeddings)
        outputs = discriminator(fake_images.detach(), text_embeddings)
        d_loss_fake = criterion(outputs, fake_labels)


        # Backpropagate and update discriminator
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()

        # Train generator
        optimizer_g.zero_grad()
        outputs = discriminator(fake_images, text_embeddings)
        g_loss = criterion(outputs, real_labels)

        # Backpropagate and update generator
        g_loss.backward()
        optimizer_g.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

        torch.save(generator.state_dict(), f"/content/drive/MyDrive/Material_Project_Data/generator_epoch_{epoch+1}.pth")
        torch.save(discriminator.state_dict(), f"/content/drive/MyDrive/Material_Project_Data/discriminator_epoch_{epoch+1}.pth")


print("Training complete!")
